--- Day 16: Packet Decoder ---

As you leave the cave and reach open waters, you receive a transmission from the Elves back on the ship.

The transmission was sent using the Buoyancy Interchange Transmission System (BITS), a method of packing numeric expressions into a binary sequence. Your submarine's computer has saved the transmission in hexadecimal (your puzzle input).

The first step of decoding the message is to convert the hexadecimal representation into binary. Each character of hexadecimal corresponds to four bits of binary data:

```
0 = 0000
1 = 0001
2 = 0010
3 = 0011
4 = 0100
5 = 0101
6 = 0110
7 = 0111
8 = 1000
9 = 1001
A = 1010
B = 1011
C = 1100
D = 1101
E = 1110
F = 1111
```

The BITS transmission contains a single packet at its outermost layer which itself contains many other packets. The hexadecimal representation of this packet might encode a few extra 0 bits at the end; these are not part of the transmission and should be ignored.

Every packet begins with a standard header: the first three bits encode the packet version, and the next three bits encode the packet type ID. These two values are numbers; all numbers encoded in any packet are represented as binary with the most significant bit first. For example, a version encoded as the binary sequence 100 represents the number 4.

Packets with type ID 4 represent a literal value. Literal value packets encode a single binary number. To do this, the binary number is padded with leading zeroes until its length is a multiple of four bits, and then it is broken into groups of four bits. Each group is prefixed by a 1 bit except the last group, which is prefixed by a 0 bit. These groups of five bits immediately follow the packet header. For example, the hexadecimal string D2FE28 becomes:

```
110100101111111000101000
VVVTTTAAAAABBBBBCCCCC
```

Below each bit is a label indicating its purpose:

The three bits labeled V (110) are the packet version, 6.
The three bits labeled T (100) are the packet type ID, 4, which means the packet is a literal value.
The five bits labeled A (10111) start with a 1 (not the last group, keep reading) and contain the first four bits of the number, 0111.
The five bits labeled B (11110) start with a 1 (not the last group, keep reading) and contain four more bits of the number, 1110.
The five bits labeled C (00101) start with a 0 (last group, end of packet) and contain the last four bits of the number, 0101.
The three unlabeled 0 bits at the end are extra due to the hexadecimal representation and should be ignored.
So, this packet represents a literal value with binary representation 011111100101, which is 2021 in decimal.

Every other type of packet (any packet with a type ID other than 4) represent an operator that performs some calculation on one or more sub-packets contained within. Right now, the specific operations aren't important; focus on parsing the hierarchy of sub-packets.

An operator packet contains one or more packets. To indicate which subsequent binary data represents its sub-packets, an operator packet can use one of two modes indicated by the bit immediately after the packet header; this is called the length type ID:

If the length type ID is 0, then the next 15 bits are a number that represents the total length in bits of the sub-packets contained by this packet.
If the length type ID is 1, then the next 11 bits are a number that represents the number of sub-packets immediately contained by this packet.
Finally, after the length type ID bit and the 15-bit or 11-bit field, the sub-packets appear.

For example, here is an operator packet (hexadecimal string 38006F45291200) with length type ID 0 that contains two sub-packets:

```
00111000000000000110111101000101001010010001001000000000
VVVTTTILLLLLLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBBBBBB
```

The three bits labeled V (001) are the packet version, 1.
The three bits labeled T (110) are the packet type ID, 6, which means the packet is an operator.
The bit labeled I (0) is the length type ID, which indicates that the length is a 15-bit number representing the number of bits in the sub-packets.
The 15 bits labeled L (000000000011011) contain the length of the sub-packets in bits, 27.
The 11 bits labeled A contain the first sub-packet, a literal value representing the number 10.
The 16 bits labeled B contain the second sub-packet, a literal value representing the number 20.
After reading 11 and 16 bits of sub-packet data, the total length indicated in L (27) is reached, and so parsing of this packet stops.

As another example, here is an operator packet (hexadecimal string EE00D40C823060) with length type ID 1 that contains three sub-packets:

```
11101110000000001101010000001100100000100011000001100000
VVVTTTILLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBCCCCCCCCCCC
```

The three bits labeled V (111) are the packet version, 7.
The three bits labeled T (011) are the packet type ID, 3, which means the packet is an operator.
The bit labeled I (1) is the length type ID, which indicates that the length is a 11-bit number representing the number of sub-packets.
The 11 bits labeled L (00000000011) contain the number of sub-packets, 3.
The 11 bits labeled A contain the first sub-packet, a literal value representing the number 1.
The 11 bits labeled B contain the second sub-packet, a literal value representing the number 2.
The 11 bits labeled C contain the third sub-packet, a literal value representing the number 3.
After reading 3 complete sub-packets, the number of sub-packets indicated in L (3) is reached, and so parsing of this packet stops.

For now, parse the hierarchy of the packets throughout the transmission and add up all of the version numbers.

Here are a few more examples of hexadecimal-encoded transmissions:

`8A004A801A8002F478` represents an operator packet (version 4) which contains an operator packet (version 1) which contains an operator packet (version 5) which contains a literal value (version 6); this packet has a version sum of 16.
`620080001611562C8802118E34` represents an operator packet (version 3) which contains two sub-packets; each sub-packet is an operator packet that contains two literal values. This packet has a version sum of 12.
`C0015000016115A2E0802F182340` has the same structure as the previous example, but the outermost packet uses a different length type ID. This packet has a version sum of 23.
`A0016C880162017C3686B18A3D4780` is an operator packet that contains an operator packet that contains an operator packet that contains five literal values; it has a version sum of 31.
Decode the structure of your hexadecimal-encoded BITS transmission; what do you get if you add up the version numbers in all packets?

In [44]:
inp = "D2FE28"

In [120]:
inp = """
E0529D18025800ABCA6996534CB22E4C00FB48E233BAEC947A8AA010CE1249DB51A02CC7DB67EF33D4002AE6ACDC40101CF0449AE4D9E4C071802D400F84BD21CAF3C8F2C35295EF3E0A600848F77893360066C200F476841040401C88908A19B001FD35CCF0B40012992AC81E3B980553659366736653A931018027C87332011E2771FFC3CEEC0630A80126007B0152E2005280186004101060C03C0200DA66006B8018200538012C01F3300660401433801A6007380132DD993100A4DC01AB0803B1FE2343500042E24C338B33F5852C3E002749803B0422EC782004221A41A8CE600EC2F8F11FD0037196CF19A67AA926892D2C643675A0C013C00CC0401F82F1BA168803510E3942E969C389C40193CFD27C32E005F271CE4B95906C151003A7BD229300362D1802727056C00556769101921F200AC74015960E97EC3F2D03C2430046C0119A3E9A3F95FD3AFE40132CEC52F4017995D9993A90060729EFCA52D3168021223F2236600ECC874E10CC1F9802F3A71C00964EC46E6580402291FE59E0FCF2B4EC31C9C7A6860094B2C4D2E880592F1AD7782992D204A82C954EA5A52E8030064D02A6C1E4EA852FE83D49CB4AE4020CD80272D3B4AA552D3B4AA5B356F77BF1630056C0119FF16C5192901CEDFB77A200E9E65EAC01693C0BCA76FEBE73487CC64DEC804659274A00CDC401F8B51CE3F8803B05217C2E40041A72E2516A663F119AC72250A00F44A98893C453005E57415A00BCD5F1DD66F3448D2600AC66F005246500C9194039C01986B317CDB10890C94BF68E6DF950C0802B09496E8A3600BCB15CA44425279539B089EB7774DDA33642012DA6B1E15B005C0010C8C917A2B880391160944D30074401D845172180803D1AA3045F00042630C5B866200CC2A9A5091C43BBD964D7F5D8914B46F040
""".strip()

In [122]:
from dataclasses import dataclass, asdict
import typing as tp
from pprint import pprint
from math import prod

In [146]:
@dataclass
class Packet:
    binary: str
    version: int
    type_id: int
    is_operator: bool
    total_length: int

    literal: tp.Optional[int]
    length_type_id: tp.Optional[int]
    subpackets: list['Packet']

    def __init__(self, binary_input: str):
#         print('===\nParsing packet:', binary_input)                
        self.version = int(binary_input[:3], 2)
        self.type_id = int(binary_input[3:6], 2)
        self.binary = binary_input[:6]
        
        self.is_operator = self.type_id != 4
        
        # literal-specific
        self.literal = None
        
        # operator-specific
        self.length_type_id = None        
        self.subpackets = []
        
        if not self.is_operator:
            self.parse_literal(binary_input[6:])
        else:
            self.parse_operator(binary_input[6:])
            
        self.total_length = len(self.binary)
                        
    def parse_literal(self, data: str):
#         print('Parsing literal', data)
        literal_numbers = ''
        
        for offset in range(500):  # large enough
            chunk = data[offset * 5: offset * 5 + 5]
#             print('Literal chunk: ', chunk)            
            self.binary += chunk
            
            should_continue = int(chunk[0])            
            literal_numbers += chunk[1:] 
            if not should_continue:
                break
            
        self.literal = int(literal_numbers, 2)
        
    def parse_operator(self, data: str):
        self.length_type_id = int(data[0])
        self.binary += data[0]
        data = data[1:]
        
        if self.length_type_id == 0:
            # the length is a 15-bit number representing the number of bits in the sub-packets.
#             print('LTID=0, taking next 15 bits')
            subpackets_length = int(data[:15], 2)
#             print('Subpackets len:', data[:15], '=', subpackets_length)            
            self.binary += data[:15]
            data = data[15:15 + subpackets_length]
#             print('Subpackets data', data)            

            while len(data) > 7:                
                subp = Packet(data)
                self.binary += data[:subp.total_length]
                data = data[subp.total_length:]
                self.subpackets.append(subp)

        else:
            # length is a 11-bit number representing the number of sub-packets.
#             print('LTID=1, taking next 11 bits')
            subpackets_count = int(data[:11], 2)
#             print('Subpackets cnt:', data[:11], '=', subpackets_count)
            self.binary += data[:11]
            data = data[11:]
#             print('Subpackets cdat', data)            
            
            for _ in range(subpackets_count):
                subp = Packet(data)
                self.binary += data[:subp.total_length]
                data = data[subp.total_length:]
                self.subpackets.append(subp)

    def version_sum(self):
        return self.version + sum([sp.version_sum() for sp in self.subpackets], 0)
                
    def calculate(self):
        sub_values = [p.calculate() for p in self.subpackets]
        
        if self.type_id == 0:
            return sum(sub_values, 0)
        
        if self.type_id == 1:
            return prod(sub_values)
        
        if self.type_id == 2:
            return min(sub_values)
        
        if self.type_id == 3:
            return max(sub_values)

        if self.type_id == 4:
            return self.literal
        
        # remaining are binary operators
        a, b = sub_values
        
        if self.type_id == 5:
            return int(a > b)
        
        if self.type_id == 6:
            return int(a < b)

        if self.type_id == 7:
            return int(a == b)
        
                
def solve12(data: str):
    packet_binary_string = ''.join([
        '%04d' % (int(bin(int(letter, 16))[2:]))
        for letter in data
    ])
    
#     print(f'Incoming binary:\n{packet_binary_string}\n')
    
    p = Packet(packet_binary_string)    
    print()
    print('Version sum:', p.version_sum())
#     pprint(asdict(p))    
    print('Expression result', p.calculate())
    

solve12(inp)
# solve12('9C0141080250320F1802104A08')


Version sum: 999
Expression result 3408662834145


--- Part Two ---

Now that you have the structure of your transmission decoded, you can calculate the value of the expression it represents.

Literal values (type ID 4) represent a single number as described above. The remaining type IDs are more interesting:

- Packets with type ID 0 are sum packets - their value is the sum of the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 1 are product packets - their value is the result of multiplying together the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 2 are minimum packets - their value is the minimum of the values of their sub-packets.
- Packets with type ID 3 are maximum packets - their value is the maximum of the values of their sub-packets.
- Packets with type ID 5 are greater than packets - their value is 1 if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 6 are less than packets - their value is 1 if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 7 are equal to packets - their value is 1 if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.

Using these rules, you can now work out the value of the outermost packet in your BITS transmission.

For example:

- C200B40A82 finds the sum of 1 and 2, resulting in the value 3.
- 04005AC33890 finds the product of 6 and 9, resulting in the value 54.
- 880086C3E88112 finds the minimum of 7, 8, and 9, resulting in the value 7.
- CE00C43D881120 finds the maximum of 7, 8, and 9, resulting in the value 9.
- D8005AC2A8F0 produces 1, because 5 is less than 15.
- F600BC2D8F produces 0, because 5 is not greater than 15.
- 9C005AC2F8F0 produces 0, because 5 is not equal to 15.
- 9C0141080250320F1802104A08 produces 1, because 1 + 3 = 2 * 2.

What do you get if you evaluate the expression represented by your hexadecimal-encoded BITS transmission?



24